# SNS SCRAPE

### Install SNS Scrape

In [1]:
!pip install snscrape

### Setup SNS Scrape for our Tweets

In [98]:
import snscrape.modules.twitter as sntwitter
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd


def run_the_tweet(keywords,incident_date, n_months, min_retweets, language):
    date_format = '%Y-%m-%d'
    dtObj = datetime.strptime(incident_date, date_format)
    
    begin_date = (dtObj - relativedelta(months=n_months)).date()
    since_date = (dtObj - relativedelta(months=n_months)).date()
    
    until_date = since_date + timedelta(days=1)
    final_date = (dtObj + relativedelta(months=n_months)).date()
    
    n_days = (final_date-since_date).days
    
    query = " OR ".join(keywords)
    if len(keywords) == 1:
        query = keywords[0]
    tweets_list= []
    for x in range(n_days): #no. of days
        for i,tweet in enumerate(sntwitter.TwitterSearchScraper(f'{query} since:{since_date.strftime(date_format)} until:{until_date.strftime(date_format)}  min_retweets:{min_retweets} lang:{language}').get_items()):
            if i > 50: #no. of output tweet
                break
            else:
                tweets_list.append([tweet.date, tweet.retweetCount, tweet.likeCount, tweet.content, tweet.user.username]) #append if statement satisfy
        if x < n_days : # no. of days you want to be return
            since_date = since_date + timedelta(days=1) #add another day
            until_date = until_date + timedelta(days=1) #add another day
        else:
            break
    tweets_df = pd.DataFrame(tweets_list, columns=['Datetime',"retweet","likes", 'Text', 'Username'])
    return tweets_df

### Clean Tweets

In [3]:
!pip install nltk

In [4]:
!pip install emot

In [5]:
!pip install emoji

In [6]:
pip install clean-text

Note: you may need to restart the kernel to use updated packages.


In [99]:
import re
import string
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import emoji
from cleantext import clean


import string
import re

from emot.emo_unicode import UNICODE_EMOJI

def clean_text(text):
    #changing to lowercase
    text = text.lower()
    
    # removing #´s 
    text = re.sub(r'#[A-Za-z0-9]+', '', text)
    text = re.sub(r'#', '', text)
    
    #remove RT
    text = re.sub(r'RT[\s]+', '', text)
    
    #remove links
    text = re.sub(r'https?:\/\/\S+', '', text)
    text = re.sub(r"www.\S+", "", text)
    
    #remove indentation
    text = re.sub(r'\n', '', text)

    #remove emojis
    text = clean(text, no_emoji=True)
    
    
    #tokenizing words
    tokens = word_tokenize(text)
    
    filtered_words = [w for w in tokens if w not in list(UNICODE_EMOJI.keys())]
    
    return text

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/stefantonto/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/stefantonto/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


***
# SENTIMENT ANALYSIS

In [9]:
!pip install transformers

In [10]:
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.8/137.8 MB 3.7 MB/s eta 0:00:0000:0100:01


In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

/Users/stefantonto/.pyenv/versions/3.8.12/envs/cancelled/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification mod

In [33]:
from transformers import pipeline

sentiment_task = pipeline("sentiment-analysis", model= model, tokenizer= tokenizer, top_k=3)

In [34]:
chris_brown_tweets = run_the_tweet(["chris brown"], "2022-08-07" ,1, "10", "en")

In [35]:
chris_brown_tweets['cleaned_tweet'] = chris_brown_tweets.Text.apply(clean_text)

In [36]:
chris_brown_tweets

,Datetime,retweet,likes,Text,Username,cleaned_tweet
0,2022-07-07 23:59:21+00:00,40,159,"Davido owe Shakespeare nothing, absolutely NOT...",30BG_CEO,"davido owe shakespeare nothing, absolutely not..."
1,2022-07-07 23:58:01+00:00,159,771,Davido and Chris brown is like Boli and ground...,Tobiloba_mide,davido and chris brown is like boli and ground...
2,2022-07-07 23:57:17+00:00,182,1045,Davido and Chris brown no Dey miss🔥🚨,malcom_xo,davido and chris brown no dey miss
3,2022-07-08 23:41:28+00:00,57,413,"Davido, Chris Brown &amp; Nobody Has To Know i...",lifeofolaa,"davido, chris brown & nobody has to know is tr..."
4,2022-07-08 23:38:02+00:00,97,776,Chris brown ft Davido Nobody has to know is 🙌🏻...,Amara_Layo,chris brown ft davido nobody has to know is ev...
...,...,...,...,...,...,...
181,2022-09-05 21:24:41+00:00,29,38,"Jay bahd be weed smoker, Philan be mmaa p3 kil...",kwesi_zitojnr,"jay bahd be weed smoker, philan be mmaa p3 kil..."
182,2022-09-05 20:51:42+00:00,16,95,|🚨\nExtra Spotify Update — Streams\n\n•Time N ...,RemaStats,|extra spotify update streamstime n affection ...
183,2022-09-06 22:23:42+00:00,26,104,I don't know if I am the only one but I feel “...,RabsonLee,"i don't know if i am the only one but i feel ""..."
184,2022-09-06 21:50:24+00:00,38,188,The Goat @davido and his fellow American Goat ...,30BG_CEO,the goat @davido and his fellow american goat ...


In [100]:
ts_tweets = run_the_tweet(["Taylor Swift"], "2022-07-31" ,1, "10", "en")

In [101]:
ts_tweets

,Datetime,retweet,likes,Text,Username
0,2022-06-30 23:47:21+00:00,49,246,Most streamed artists on Global Spotify (June ...,WORLDMUSICAWARD
1,2022-06-30 23:36:15+00:00,1065,12458,Taylor Swift is engaged to Joe Alwyn: report h...,nypost
2,2022-06-30 23:20:45+00:00,51,826,does it ever hit you that phoebe bridgers and ...,phoebesmoons
3,2022-06-30 22:58:03+00:00,1133,8686,me?????? obsessed?????? with????? Carolina by ...,klbegley15
4,2022-06-30 22:53:20+00:00,11,180,I miss watching updates about american weather...,icantunseeit
...,...,...,...,...,...
3157,2022-08-30 20:56:09+00:00,11,151,"📸 | @SabrinaAnnLynn, @taylorswift13, and Rosé ...",sabrinaupdatehq
3158,2022-08-30 20:54:36+00:00,29,210,"I'd rather listen to ""All Summer Long"" by Kid ...",NickAdamsinUSA
3159,2022-08-30 20:53:15+00:00,16,87,Swifties who openly use their account to mock ...,oneswiftlover
3160,2022-08-30 20:51:16+00:00,145,2161,‼️UPDATE: @taylorswift13's profile picture has...,TheSwiftSociety


In [102]:
ts_tweets['cleaned_tweet'] = ts_tweets.Text.apply(clean_text)

In [103]:
def get_sentiment_score_dictionary(df_cleanedtext):
    list_sentiments = []
    for tweet in ts_tweets.cleaned_tweet:
        sentiment_task_result = sentiment_task(tweet)[0]
        tweet_sentiment = {}
        for label in sentiment_task_result:
            if label['label'] == 'Neutral':
                tweet_sentiment['Neutral'] = label['score']
            elif label['label'] == 'Positive':
                tweet_sentiment['Positive'] = label['score']
            else:
                tweet_sentiment['Negative'] = label['score']
        list_sentiments.append(tweet_sentiment)
    return list_sentiments

def get_sentiment_score_classification(list_sentiments):
    sentiment_classifications = []
    for list_sentiment in list_sentiments:
        if (list_sentiment['Positive'] > list_sentiment['Negative']) and (list_sentiment['Positive'] > list_sentiment['Neutral']):
            sentiment_classifications.append('Positive')
        elif (list_sentiment['Neutral'] > list_sentiment['Negative']) and (list_sentiment['Positive'] < list_sentiment['Neutral']):
            sentiment_classifications.append('Neutral')
        else:
            sentiment_classifications.append('Negative')
    return sentiment_classifications

In [104]:
classified_sentiment_ts = get_sentiment_score_classification(get_sentiment_score_dictionary(ts_tweets.cleaned_tweet))

In [105]:
pd.merge(ts_tweets, pd.DataFrame.from_dict(get_sentiment_score_dictionary(ts_tweets.cleaned_tweet)), left_index= True, right_index= True)

,Datetime,retweet,likes,Text,Username,cleaned_tweet,Neutral,Positive,Negative
0,2022-06-30 23:47:21+00:00,49,246,Most streamed artists on Global Spotify (June ...,WORLDMUSICAWARD,most streamed artists on global spotify (june ...,0.733290,0.256657,0.010054
1,2022-06-30 23:36:15+00:00,1065,12458,Taylor Swift is engaged to Joe Alwyn: report h...,nypost,taylor swift is engaged to joe alwyn: report,0.955908,0.024010,0.020082
2,2022-06-30 23:20:45+00:00,51,826,does it ever hit you that phoebe bridgers and ...,phoebesmoons,does it ever hit you that phoebe bridgers and ...,0.724014,0.260200,0.015786
3,2022-06-30 22:58:03+00:00,1133,8686,me?????? obsessed?????? with????? Carolina by ...,klbegley15,me?????? obsessed?????? with????? carolina by ...,0.040586,0.956709,0.002705
4,2022-06-30 22:53:20+00:00,11,180,I miss watching updates about american weather...,icantunseeit,i miss watching updates about american weather...,0.670205,0.159175,0.170620
...,...,...,...,...,...,...,...,...,...
3157,2022-08-30 20:56:09+00:00,11,151,"📸 | @SabrinaAnnLynn, @taylorswift13, and Rosé ...",sabrinaupdatehq,"| @sabrinaannlynn, @taylorswift13, and rose fr...",0.841150,0.155392,0.003458
3158,2022-08-30 20:54:36+00:00,29,210,"I'd rather listen to ""All Summer Long"" by Kid ...",NickAdamsinUSA,"i'd rather listen to ""all summer long"" by kid ...",0.280083,0.043955,0.675962
3159,2022-08-30 20:53:15+00:00,16,87,Swifties who openly use their account to mock ...,oneswiftlover,swifties who openly use their account to mock ...,0.167535,0.010010,0.822455
3160,2022-08-30 20:51:16+00:00,145,2161,‼️UPDATE: @taylorswift13's profile picture has...,TheSwiftSociety,update: @taylorswift13's profile picture has b...,0.893864,0.100866,0.005270


In [106]:
ts_df = pd.merge(ts_tweets, pd.DataFrame.from_dict(classified_sentiment_ts), left_index= True, right_index= True)

In [107]:
ts_df.rename(columns = {0:'sentiment_classification'}, inplace = True)

In [108]:
ts_df.sentiment_classification.value_counts()

Neutral     1700
Positive    1260
Negative     202
Name: sentiment_classification, dtype: int64

In [109]:
ts_df

,Datetime,retweet,likes,Text,Username,cleaned_tweet,sentiment_classification
0,2022-06-30 23:47:21+00:00,49,246,Most streamed artists on Global Spotify (June ...,WORLDMUSICAWARD,most streamed artists on global spotify (june ...,Neutral
1,2022-06-30 23:36:15+00:00,1065,12458,Taylor Swift is engaged to Joe Alwyn: report h...,nypost,taylor swift is engaged to joe alwyn: report,Neutral
2,2022-06-30 23:20:45+00:00,51,826,does it ever hit you that phoebe bridgers and ...,phoebesmoons,does it ever hit you that phoebe bridgers and ...,Neutral
3,2022-06-30 22:58:03+00:00,1133,8686,me?????? obsessed?????? with????? Carolina by ...,klbegley15,me?????? obsessed?????? with????? carolina by ...,Positive
4,2022-06-30 22:53:20+00:00,11,180,I miss watching updates about american weather...,icantunseeit,i miss watching updates about american weather...,Neutral
...,...,...,...,...,...,...,...
3157,2022-08-30 20:56:09+00:00,11,151,"📸 | @SabrinaAnnLynn, @taylorswift13, and Rosé ...",sabrinaupdatehq,"| @sabrinaannlynn, @taylorswift13, and rose fr...",Neutral
3158,2022-08-30 20:54:36+00:00,29,210,"I'd rather listen to ""All Summer Long"" by Kid ...",NickAdamsinUSA,"i'd rather listen to ""all summer long"" by kid ...",Negative
3159,2022-08-30 20:53:15+00:00,16,87,Swifties who openly use their account to mock ...,oneswiftlover,swifties who openly use their account to mock ...,Negative
3160,2022-08-30 20:51:16+00:00,145,2161,‼️UPDATE: @taylorswift13's profile picture has...,TheSwiftSociety,update: @taylorswift13's profile picture has b...,Neutral


In [134]:
taylor_controversy_period = ts_df.iloc[1531:1581]

In [135]:
for x in taylor_controversy_period.cleaned_tweet:
    print(f'NEW TWEET: {x}\n')

NEW TWEET: normal people marianneconnell fancam marianne connell edit daisy edgar jones paul mescal untouchable taylor swiftcr:zcriptedz

NEW TWEET: y'all's carrds say "i don't endorse my stan's problematic actions!" until taylor swift is rightfully called out for emitting more co2 this year than an average american does in 550 years and then it's "actually, she"

NEW TWEET: that one time lana del rey asked taylor swift to make chemtrails for her music video

NEW TWEET: taylor swift:

NEW TWEET: august by taylor swift

NEW TWEET: more of selena and nicki with katy perry and taylor swift at the amas 2011

NEW TWEET: "you are what you love" ~ taylor swift

NEW TWEET: album debuts ustaylor swiftfearless - 592kspeak now - 1mred - 1.22m1989 - 1.3mreputation - 1.23mlover - 867kfolklore - 846klady gagathe fame - 24kborn this way - 174kartpop - 258kcheek to cheek - 131kjoanne - 201ka star is born - 231kchromatica - 274k

NEW TWEET: okay we got our taylor swift yearly controversy, where is the 